# Webscraping mit Scrapy

Web Scraping ist im Wesentlichen eine Form des Data Mining und bezeichnet verschiedene Methoden, um Informationen aus dem Internet zu sammeln. Im Allgemeinen wird dies mit einer Software durchgeführt, die menschliches Websurfen simuliert, um bestimmte Informationen von verschiedenen Webseiten zu sammeln. Web Scraping wird auch als Web Data Extraction, Screen Scraping oder Web Harvesting bezeichnet[1].

Eine Möglichkeit für das webscraping bietet das Python-Framework Scrapy, welches über den folgenden Befehl installiert werden kann[2]:

pip install Scrapy

Nach der Installation können wir mit dem Befehl startprojekt ein neues Projekt anlegen.
Dazu navigiert man in der Komandozeile in das Verzeichnis, in dem das Projekt angelegt werden soll[2]. 

scrapy startproject example

Die wichtigsten Dateien sind:
- items.py: Diese Datei definiert ein Modell der Felder, die gescraped werden.
- settings.py: Diese Datei definiert Einstellungen wie den Benutzeragenten und die Verzögerung zwischen den Downloadanfragen.
- spiders: In diesem Ordner wird der eigentliche Scraping- und Crawling-Code gespeichert.

Die jeweiligen Klassen können aber auch manuel in einem Notebook angelegt werden.

Sehen wir uns zunächst die Datei items an. Diese sieht standartmäßig so aus[2]:

In [1]:
import scrapy

class ExampleItem(scrapy.Item):
# define the fields for your item here like:
# name = scrapy.Field()
    pass

Für unser Beispiel sollen Daten der Seite [example.webscraping.com](http://example.webscraping.com) gesammelt werden. Die Klasse items könnte daher beispielsweise so aussehen: 

In [2]:
import scrapy

class ExampleItem(scrapy.Item):
# define the fields for your item here like:
    name = scrapy.Field()
    area = scrapy.Field()
    population = scrapy.Field()
    capital = scrapy.Field()
    

Bevor wir den Ordner Spiders betrachten. Sehen wir uns zunächst an, wie die Daten extrahiert werden können. Dies geschieht z.B. über die Anweisung response.css('title'). Das Ergebnis der Anweisung ist ein listenähnliches Objekt namens SelectorList, das eine Liste von Selector-Objekten darstellt, die XML-/HTML-Elemente umschließen und weitere Abfragen. Für Abfragen gibt es zwei Möglichkeiten [3]:
- css
- xpath

## Möglichkeit 1: CSS

Sehen wir uns als nächstes den Ordner Spiders an. Spider sind Klassen, die Scrapy verwendet, um Informationen von Webseiten zu scrappen. Sie werden von der Klasse scrapy.Spider abgeleitet und definieren die Anforderungen[2]. Für unser Beispiel wird die Klasse CountrySpieder angelegt und die Tabelle mit den Ländern mit Hilfe von css extrahiert. Zum Verständnis sollen die Daten nicht gespeichert sondern einfach ausgegeben werden.

In [3]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

class CountrySpider(scrapy.Spider):
    name = 'country'
    start_urls = ['http://example.webscraping.com/']
    allowed_domains = ['example.webscraping.com']
    
    def parse(self, response):
        # Ausgabe der SelectorList
        print(response.css('div#results div a'))
        print('')
        # Ausgabe der SelectorList mit Text
        print(response.css('div#results div a::text'))
        print('')
        # Ausgabe der Tabelle als Liste
        print(response.css('div#results div a::text').extract())
        print('')
        # Ausgabe des ersten Elements
        print(response.css('div#results div a::text')[0].extract())
        print('')
        print(response.css('div#results div a::text').extract_first())
   

runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

[<Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/Afghanista'>, <Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/Aland-Isla'>, <Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/Albania-3"'>, <Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/Algeria-4"'>, <Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/American-S'>, <Selector xpath="descendant-or-self::div[@id = 'results']/descendant-or-self::*/div/descendant-or-self::*/a" data='<a href="/places/default/view/Andorra-6"'>, <Selector xpath="descendant-or-self::div[@id 

Erläuterungen:
- name: String zur Identifizierung der Spider[2].
- start_urls: Dieses Attribut ist eine Liste von URLs zum Starten des Spiders[2].
- allowed_domains: Dieses Attribut ist eine Liste der Domänen, die gecrawlt werden können. Wenn dies nicht definiert ist, kann jede Domäne gecrawlt werden[2].
- parse: Dies ist die Standardmethode, die von Scrapy verwendet wird, um heruntergeladene Daten zu verarbeiten[4].
- CrawlerRunner:Diese Klasse ist ein dünner Wrapper, der einige einfache Helfer kapselt, um mehrere Crawler laufen zu lassen [5].
- reactor: Der Reaktor ist der Kern der Ereignisschleife. Die Ereignisschleife ist ein Programmierkonstrukt, das auf Ereignisse oder Nachrichten in einem Programm wartet und diese versendet[7]. Leider kann er nur einmal gestartet werden sonst erhält man den Fehler "ReactorNotRestartable".

Der Crawler kann aber auch über die Konsole mit fogendem Befehl gestartet werden

scrapy crawl country

Möchte man die Ergebnisse in einer Datei speichern startet man den Crawler folgendermaßen[3]:

scrapy crawl country -o test.json

Die Ergebnisse werden also in einer json-Datei gespeichert.

Das Programm gibt jedoch nur die Länder der ersten Seite aus. Möchte man die Länder aller Seiten ausgeben. Muss das Programm folgendermaßen ergänzt werden. 

In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

class CountrySpider(scrapy.Spider):
    name = 'country'
    start_urls = ['http://example.webscraping.com/']
    allowed_domains = ['example.webscraping.com']
    
    def __init__(self):
        self.download_delay = 5
    
    def parse(self, response):
        print(response.css('div#results div a::text').extract())
        print('')
        
        if response.css('div#pagination a::text').extract_first() == "Next >":
            next_page = response.css('div#pagination a::attr(href)').extract_first()
        else:
            next_page = response.css('div#pagination a::attr(href)')[1].extract()
        
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)
   

runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

[' Afghanistan', ' Aland Islands', ' Albania', ' Algeria', ' American Samoa', ' Andorra', ' Angola', ' Anguilla', ' Antarctica', ' Antigua and Barbuda']

[' Argentina', ' Armenia', ' Aruba', ' Australia', ' Austria', ' Azerbaijan', ' Bahamas', ' Bahrain', ' Bangladesh', ' Barbados']

[' Belarus', ' Belgium', ' Belize', ' Benin', ' Bermuda', ' Bhutan', ' Bolivia', ' Bonaire, Saint Eustatius and Saba', ' Bosnia and Herzegovina', ' Botswana']

[' Bouvet Island', ' Brazil', ' British Indian Ocean Territory', ' British Virgin Islands', ' Brunei', ' Bulgaria', ' Burkina Faso', ' Burundi', ' Cambodia', ' Cameroon']

[' Canada', ' Cape Verde', ' Cayman Islands', ' Central African Republic', ' Chad', ' Chile', ' China', ' Christmas Island', ' Cocos Islands', ' Colombia']

[' Comoros', ' Cook Islands', ' Costa Rica', ' Croatia', ' Cuba', ' Curacao', ' Cyprus', ' Czech Republic', ' Democratic Republic of the Congo', ' Denmark']

[' Djibouti', ' Dominica', ' Dominican Republic', ' East Timor', ' E

Als nächstes soll neben dem Namen der Länder auch die Fläche, Einwohnerzahl und die Hauptstadt zu den Ländern der ersten Seite ausgegeben werden. Dazu muss das obere Programm folgendermaßen abgeändert werden. Natürlich könnten wir auch die Informationen der Länder aller Seiten ausgeben, indem wir dem "next >"-Link folgen. Wegen der Übersicht lassen wir dies.

In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

class CountrySpider(scrapy.Spider):
    name = 'country'
    start_urls = ['http://example.webscraping.com/']
    allowed_domains = ['example.webscraping.com']
    
    def __init__(self):
        self.download_delay = 5
    
    def parse(self, response):
        urls = response.css('div#results div a::attr(href)').extract()
        for url in urls:
            url = response.urljoin(url)
            yield scrapy.Request(url=url, callback=self.parse_details)
            
    def parse_details(self, response):
        
        print("Land:", response.css('td.w2p_fw::text')[3].extract())
        print("Fläche:", response.css('td.w2p_fw::text')[0].extract())
        print("Einwohner:", response.css('td.w2p_fw::text')[1].extract())
        print("Hauptstadt:", response.css('td.w2p_fw::text')[4].extract())
        print('')
        
runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

Land: Antigua and Barbuda
Fläche: 443 square kilometres
Einwohner: 86,754
Hauptstadt: St. John's

Land: Antarctica
Fläche: 14,000,000 square kilometres
Einwohner: 0
Hauptstadt: .aq

Land: Anguilla
Fläche: 102 square kilometres
Einwohner: 13,254
Hauptstadt: The Valley

Land: Angola
Fläche: 1,246,700 square kilometres
Einwohner: 13,068,161
Hauptstadt: Luanda

Land: Andorra
Fläche: 468 square kilometres
Einwohner: 84,000
Hauptstadt: Andorra la Vella

Land: American Samoa
Fläche: 199 square kilometres
Einwohner: 57,881
Hauptstadt: Pago Pago

Land: Algeria
Fläche: 2,381,740 square kilometres
Einwohner: 34,586,184
Hauptstadt: Algiers

Land: Albania
Fläche: 28,748 square kilometres
Einwohner: 2,986,952
Hauptstadt: Tirana

Land: Aland Islands
Fläche: 1,580 square kilometres
Einwohner: 26,711
Hauptstadt: Mariehamn

Land: Afghanistan
Fläche: 647,500 square kilometres
Einwohner: 29,121,286
Hauptstadt: Kabul



## Möglichkeit 2: xpath

XPath_Ausdrücke sind zwar nicht so populär wie CSS-Selektoren, bieten aber mehr Leistungen, da sie neben dem Navigieren der Struktur auch den Inhalt betrachten können. Mit XPath kann man Dinge auswählen wie: Wählen Sie den Link, der den Text "Nächste Seite" enthält. Dadurch wird das Scraping viel einfacher [3]. Sehen wir uns das Beispiel aus Zelle X noch einmal an. Diesmal soll aber xpath statt css genutzt werden.


In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

class CountrySpider(scrapy.Spider):
    name = 'country'
    start_urls = ['http://example.webscraping.com/']
    allowed_domains = ['example.webscraping.com']
    
    def __init__(self):
        self.download_delay = 5
    
    def parse(self, response):
        urls = response.xpath('//div[@id="results"]//a/@href').extract()
        for url in urls:
            url = response.urljoin(url)
            yield scrapy.Request(url=url, callback=self.parse_details)
            
    def parse_details(self, response):
        
        print("Land:", response.xpath('//td[@class="w2p_fw"]/text()')[3].extract())
        print("Fläche:", response.xpath('//td[@class="w2p_fw"]/text()')[0].extract())
        print("Einwohner:", response.xpath('//td[@class="w2p_fw"]/text()')[1].extract())
        print("Hauptstadt:", response.xpath('//td[@class="w2p_fw"]/text()')[4].extract())
        print('')
        
runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

Land: Antigua and Barbuda
Fläche: 443 square kilometres
Einwohner: 86,754
Hauptstadt: St. John's

Land: Antarctica
Fläche: 14,000,000 square kilometres
Einwohner: 0
Hauptstadt: .aq

Land: Anguilla
Fläche: 102 square kilometres
Einwohner: 13,254
Hauptstadt: The Valley

Land: Angola
Fläche: 1,246,700 square kilometres
Einwohner: 13,068,161
Hauptstadt: Luanda

Land: Andorra
Fläche: 468 square kilometres
Einwohner: 84,000
Hauptstadt: Andorra la Vella

Land: American Samoa
Fläche: 199 square kilometres
Einwohner: 57,881
Hauptstadt: Pago Pago

Land: Algeria
Fläche: 2,381,740 square kilometres
Einwohner: 34,586,184
Hauptstadt: Algiers

Land: Albania
Fläche: 28,748 square kilometres
Einwohner: 2,986,952
Hauptstadt: Tirana

Land: Aland Islands
Fläche: 1,580 square kilometres
Einwohner: 26,711
Hauptstadt: Mariehamn

Land: Afghanistan
Fläche: 647,500 square kilometres
Einwohner: 29,121,286
Hauptstadt: Kabul



Zu Sehen ist, dass man Tags bestimmter Klassen und ids suchen kann. Mit text() wird der Text zwischen den Tags extrahiert.

Eine weitere Fähigkeit von Scrapy ist die Interaktion mit Suchfunktionen und extrahieren von Daten aus Dateien im json-Format[2]. Das folgende Beispiel sucht die ersten zehn Länder mit einem G im Namen und gibt die Ergebnisse im json-Format aus. Dazu nutzt es die Suchfunktion der Seite [example.webscraping.com](http://example.webscraping.com/places/default/search)

In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor

class CountrySpider(scrapy.Spider):
    name = 'country'
    api_url = 'http://example.webscraping.com/places/ajax/search.json?page={}&page_size=10&search_term={}'
    start_urls = [api_url.format(1, 'G')]
    allowed_domains = ['example.webscraping.com']
    
    def __init__(self):
        self.download_delay = 5
    
    def parse(self, response):
        print(response.text)
        
        
        
runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

{"records": [{"pretty_link": "<div><a href=\"/places/default/view/Bulgaria-36\"><img src=\"/places/static/images/flags/bg.png\" /> Bulgaria</a></div>", "country": "Bulgaria", "id": 2185884}, {"pretty_link": "<div><a href=\"/places/default/view/Democratic-Republic-of-the-Congo-59\"><img src=\"/places/static/images/flags/cd.png\" /> Democratic Republic of the Congo</a></div>", "country": "Democratic Republic of the Congo", "id": 2185907}, {"pretty_link": "<div><a href=\"/places/default/view/Egypt-66\"><img src=\"/places/static/images/flags/eg.png\" /> Egypt</a></div>", "country": "Egypt", "id": 2185914}, {"pretty_link": "<div><a href=\"/places/default/view/Equatorial-Guinea-68\"><img src=\"/places/static/images/flags/gq.png\" /> Equatorial Guinea</a></div>", "country": "Equatorial Guinea", "id": 2185916}, {"pretty_link": "<div><a href=\"/places/default/view/French-Guiana-77\"><img src=\"/places/static/images/flags/gf.png\" /> French Guiana</a></div>", "country": "French Guiana", "id": 21

Um die gewünschten Daten daraus zu extrahieren muss json importiert werden und in der parse-Methode extrhiert werden.

In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor
import json

class CountrySpider(scrapy.Spider):
    name = 'country'
    api_url = 'http://example.webscraping.com/places/ajax/search.json?page={}&page_size=10&search_term={}'
    start_urls = [api_url.format(1, 'G')]
    allowed_domains = ['example.webscraping.com']
    
    def __init__(self):
        self.download_delay = 5
    
    def parse(self, response):
        data = json.loads(response.text)
        
        for country in data['records']:
            print(country['country'])
        
        
runner = CrawlerRunner()
d = runner.crawl(CountrySpider)
d.addBoth(lambda _: reactor.stop())
reactor.run()

Bulgaria
Democratic Republic of the Congo
Egypt
Equatorial Guinea
French Guiana
Gabon
Gambia
Georgia
Germany
Ghana


## Quellen:
- [1] https://www.techopedia.com/definition/5212/web-scraping Letzter Aufruf am 04.06.2018
- [2] Richard Lawson. Web Scraping with Python. 2015.
- [3] https://doc.scrapy.org/en/latest/intro/tutorial.html letzter Aufruf am 04.06.2018
- [4] https://doc.scrapy.org/en/latest/topics/spiders.html#topics-spiders letzter Aufruf 05.06.2018
- [5] https://doc.scrapy.org/en/latest/topics/practices.html?highlight=crawlerRunner letzter Aufruf 05.06.2018
- [6] https://doc.scrapy.org/en/latest/topics/selectors.html#topics-selectors letzter Aufruf 05.06.2018
- [7] https://twistedmatrix.com/documents/current/core/howto/reactor-basics.html letzter Aufruf 05.06.2018